In [35]:
!pip install ofa
!pip install torch==1.4.0 torchvision==0.5.0
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

Cloning into 'IMagenet'...
remote: Enumerating objects: 120594, done.
remote: Total 120594 (delta 0), reused 0 (delta 0), pack-reused 120594
Receiving objects: 100% (120594/120594), 212.68 MiB | 26.71 MiB/s, done.
Resolving deltas: 100% (1115/1115), done.
Checking out files: 100% (120206/120206), done.
test  train  val  wnids.txt  words.txt


In [ ]:
import time
import skimage.io as nd
import numpy as np

path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict
  
def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])      
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [nd.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i))) for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(nd.imread( path + 'val/images/{}'.format(img_name)))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)
  
train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())

starting loading data


In [8]:
import copy

import torch

from ofa.model_zoo import ofa_net

raw_resnet = ofa_net('ofa_resnet50', pretrained=True)
raw_resnet.set_max_net()
max_subnet = raw_resnet.get_active_subnet()
raw_resnet.set_active_subnet(d=0, w=0)
min_subnet = raw_resnet.get_active_subnet()
print(max_subnet.grouped_block_index)
print(min_subnet.grouped_block_index)


[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11, 12, 13], [14, 15, 16, 17]]
[[0, 1], [2, 3], [4, 5, 6, 7], [8, 9]]
8


In [10]:
max_subnet.eval()
max_subnet.qconfig = torch.quantization.default_qconfig
min_subnet.eval()
min_subnet.qconfig = torch.quantization.default_qconfig


In [34]:
from torch.nn import Conv2d, ReLU, BatchNorm2d, Linear
max_subnet = torch.quantization.fuse_modules(max_subnet,  ['Conv2d', 'ReLU'])


AttributeError: ignored

In [11]:
max_subnet_1 = torch.quantization.prepare(max_subnet)
min_subnet_1 = torch.quantization.prepare(min_subnet)

In [12]:
max_q1 = torch.quantization.convert(max_subnet_1, inplace=True)
min_q1 = torch.quantization.convert(min_subnet_1, inplace=True)

/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:172: UserWarning: Must run observer before calling calculate_qparams.                           Returning default scale and zero point.
  Returning default scale and zero point.")


In [18]:
max_mods = dict(max_q1.named_children())
print(max_mods['input_stem'][-1])
print(max_mods['blocks'][2])
min_mods = dict(min_q1.named_children())
print(min_mods['input_stem'][-1])
print(min_mods['blocks'][2])
# print(len(min_mods['blocks']))


ConvLayer(
  (conv): QuantizedConv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): QuantizedReLU(inplace=True)
)
ResNetBottleneckBlock(
  (conv1): Sequential(
    (conv): QuantizedConv2d(256, 88, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0, bias=False)
    (bn): BatchNorm2d(88, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): QuantizedReLU(inplace=True)
  )
  (conv2): Sequential(
    (conv): QuantizedConv2d(88, 88, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1), bias=False)
    (bn): BatchNorm2d(88, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): QuantizedReLU(inplace=True)
  )
  (conv3): Sequential(
    (conv): QuantizedConv2d(88, 256, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0, bias=False)
    (bn): BatchNorm2d(256, eps=1e-05, mo

In [29]:
x = torch.tensor([[-1.0, 0.0], [1.0, 2.0],  [1.0, 2.0]])
print(x.shape)
print(torch.quantize_per_channel(x, torch.tensor([0.1, 0.01, 0.01]), torch.tensor([10, 0, 0]), 0, torch.quint8).int_repr())


torch.Size([3, 2])
tensor([[  0,  10],
        [100, 200],
        [100, 200]], dtype=torch.uint8)
